In [6]:
import pandas as pd
from sqlalchemy import create_engine
from DATA.stock_invest_function import *


In [9]:
def add_yoy_growth(df, value_column='value', group_column='root_hs_code', date_column='date'):
    """
    root_hs_code별로 value 컬럼의 연간 증가율을 계산하여 새로운 컬럼으로 추가합니다.
    """
    df = df.copy()
    df[date_column] = pd.to_datetime(df[date_column])
    df.sort_values(by=[group_column, date_column], inplace=True)

    # YoY (12개월 전 대비 비율 변화율) 계산
    df[f'{value_column}_yoy'] = (
        df.groupby(group_column)[value_column]
        .transform(lambda x: x.pct_change(periods=12))
    )

    return df

In [19]:
db_info = {
    'user': 'stox7412',         # 예: 'root'
    'password': 'Apt106503!~', # 예: '1234'
    'host': 'hystox74.synology.me',         # 예: 'localhost' 또는 IP
    'port': '3307',              # 기본 포트는 보통 3306
    'database': 'investar'        # 예: 'trade_data'
}

trade_df = fetch_table_data(db_info, "korea_monthly_trade_act_forecast_data")

df_combined_with_yoy = add_yoy_growth(trade_df)

company_num = 490

# 1. 예측 데이터만 필터링하고 NaN 제거
df_forecast_only = df_combined_with_yoy[
    (df_combined_with_yoy['is_forecast'] == 1) &
    (df_combined_with_yoy['value_yoy'].notna())
]

# 2. root_hs_code별 평균 YoY 증가율 계산
avg_yoy_by_code = (
    df_forecast_only
    .groupby('root_hs_code')['value_yoy']
    .mean()
    .reset_index()
    .rename(columns={'value_yoy': 'avg_forecast_yoy'})
)

# 3. 수출 증가율 기준으로 상위 10개 추출
top_company_codes = avg_yoy_by_code.sort_values(by='avg_forecast_yoy', ascending=False).head(company_num)

✅ 'korea_monthly_trade_act_forecast_data' 테이블에서 121600건의 데이터를 가져왔습니다.


In [20]:
top_company_codes

,root_hs_code,avg_forecast_yoy
302,841940,14.327663
390,851419,3.052754
336,845899,2.902217
476,890120,1.118835
301,841939,0.948747
...,...,...
243,740321,-0.133070
411,852990,-0.133826
167,621710,-0.134349
481,900211,-0.136224


In [12]:
## 강소기업 데이터 가져오기

In [21]:
company_df = fetch_table_data(db_info, 'hs_code_by_kr_monster_company')
company_df.rename(columns={'hs_code_6d': 'root_hs_code'}, inplace=True)

✅ 'hs_code_by_kr_monster_company' 테이블에서 185건의 데이터를 가져왔습니다.


In [22]:
pd.merge(company_df, top_company_codes, on='root_hs_code', how='left', indicator=True)

,hs_code,품목명,Code,Name,root_hs_code,avg_forecast_yoy,_merge
0,392321,엘씨디카세트,A089980,상아프론테크,392321,0.089682,both
1,470710,폐골판지,A016590,신대양제지,470710,0.067065,both
2,480511,골심지,A016590,신대양제지,480511,0.159842,both
3,560394,폴리우레탄합성피혁,A035150,백산,560394,-0.029331,both
4,630293,극세사클리너,A065950,웰크론,630293,0.012888,both
...,...,...,...,...,...,...,...
180,9031809091,반도체오버레이계측기,A322310,오로스테크놀로지,903180,0.016559,both
181,9401309000,디지털구강내엑스선영상획득장치,A228850,레이언스,940130,NaN,left_only
182,9403309000,사무용의자,A016800,퍼시스,940330,0.046395,both
183,9405409000,선박용형광등기구,A108380,대양전기공업,940540,NaN,left_only


In [16]:
top_company_codes

,root_hs_code,avg_forecast_yoy
302,841940,14.327663
390,851419,3.052754
336,845899,2.902217
476,890120,1.118835
301,841939,0.948747
...,...,...
243,740321,-0.133070
411,852990,-0.133826
167,621710,-0.134349
481,900211,-0.136224
